In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('logs/formula_generation_fixed.csv')

In [3]:
from collections import Counter

In [4]:
def count_entities(formula):
    return Counter(formula)['e']

def count_projections(formula):
    return Counter(formula)['p']

In [5]:
df['original_e_count'] = df.original.map(count_entities)

In [6]:
df

,original,DeMorgan,DNF,diff,DNF+diff,DNF+MultiIU,DNF+MultiIUD,formula_id,original_e_count
0,"(p,(e))","(p,(e))","(p,(e))","(p,(e))","(p,(e))","(p,(e))","(p,(e))",type0000,1
1,"(p,(p,(e)))","(p,(p,(e)))","(p,(p,(e)))","(p,(p,(e)))","(p,(p,(e)))","(p,(p,(e)))","(p,(p,(e)))",type0001,1
2,"(p,(p,(p,(e))))","(p,(p,(p,(e))))","(p,(p,(p,(e))))","(p,(p,(p,(e))))","(p,(p,(p,(e))))","(p,(p,(p,(e))))","(p,(p,(p,(e))))",type0002,1
3,"(p,(i,(p,(p,(e))),(p,(p,(e)))))","(p,(i,(p,(p,(e))),(p,(p,(e)))))","(p,(i,(p,(p,(e))),(p,(p,(e)))))","(p,(i,(p,(p,(e))),(p,(p,(e)))))","(p,(i,(p,(p,(e))),(p,(p,(e)))))","(p,(i,(p,(p,(e))),(p,(p,(e)))))","(p,(i,(p,(p,(e))),(p,(p,(e)))))",type0003,2
4,"(p,(i,(n,(p,(e))),(p,(p,(e)))))","(p,(i,(n,(p,(e))),(p,(p,(e)))))","(p,(i,(n,(p,(e))),(p,(p,(e)))))","(p,(d,(p,(p,(e))),(p,(e))))","(p,(d,(p,(p,(e))),(p,(e))))","(p,(i,(n,(p,(e))),(p,(p,(e)))))","(p,(D,(p,(e)),(p,(p,(e)))))",type0004,2
...,...,...,...,...,...,...,...,...,...
246,"(u,(p,(e)),(u,(p,(p,(e))),(p,(p,(p,(e))))))","(n,(i,(n,(n,(i,(n,(p,(p,(e)))),(n,(p,(p,(p,(e)...","(u,(p,(e)),(u,(p,(p,(e))),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(p,(e))),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(p,(e))),(p,(p,(p,(e))))))","(U,(p,(e)),(p,(p,(e))),(p,(p,(p,(e)))))","(U,(p,(e)),(p,(p,(e))),(p,(p,(p,(e)))))",type0456,3
247,"(u,(p,(e)),(u,(p,(e)),(p,(p,(e)))))","(n,(i,(n,(n,(i,(n,(p,(e))),(n,(p,(p,(e))))))),...","(u,(p,(e)),(u,(p,(e)),(p,(p,(e)))))","(u,(p,(e)),(u,(p,(e)),(p,(p,(e)))))","(u,(p,(e)),(u,(p,(e)),(p,(p,(e)))))","(U,(p,(e)),(p,(e)),(p,(p,(e))))","(U,(p,(e)),(p,(e)),(p,(p,(e))))",type0457,3
248,"(u,(p,(e)),(u,(p,(p,(p,(e)))),(p,(p,(p,(e))))))","(n,(i,(n,(n,(i,(n,(p,(p,(p,(e))))),(n,(p,(p,(p...","(u,(p,(e)),(u,(p,(p,(p,(e)))),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(p,(p,(e)))),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(p,(p,(e)))),(p,(p,(p,(e))))))","(U,(p,(e)),(p,(p,(p,(e)))),(p,(p,(p,(e)))))","(U,(p,(e)),(p,(p,(p,(e)))),(p,(p,(p,(e)))))",type0459,3
249,"(u,(p,(e)),(u,(p,(e)),(p,(p,(p,(e))))))","(n,(i,(n,(n,(i,(n,(p,(e))),(n,(p,(p,(p,(e)))))...","(u,(p,(e)),(u,(p,(e)),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(e)),(p,(p,(p,(e))))))","(u,(p,(e)),(u,(p,(e)),(p,(p,(p,(e))))))","(U,(p,(e)),(p,(e)),(p,(p,(p,(e)))))","(U,(p,(e)),(p,(e)),(p,(p,(p,(e)))))",type0460,3


In [7]:
from fol.foq_v2 import parse_formula

In [8]:
def count_query_depth(query):
    if query.__o__ == 'e':
        return 0
    elif query.__o__ in 'uiUID':
        return max(count_query_depth(q) for q in query.sub_queries)
    elif query.__o__ == 'p':
        return count_query_depth(query.query) + 1
    elif query.__o__ == 'n':
        return count_query_depth(query.query)
    elif query.__o__ == 'd':
        return max(count_query_depth(query.lquery), count_query_depth(query.rquery))
    else:
        raise NotImplementedError

In [9]:
def count_depth(formula):
    try:
        query = parse_formula(formula)
    except:
        print(formula)
    return count_query_depth(query)

In [10]:
forms = ['original', 'DeMorgan', 'DNF', 'diff', 'DNF+diff', 'DNF+MultiIU', 'DNF+MultiIUD']

In [11]:
for f in forms:
    df[f"{f}:e_counts"] = df[f].map(count_entities)
    df[f"{f}:p_counts"] = df[f].map(count_projections)
    df[f"{f}:p_depth"] = df[f].map(count_depth)

In [12]:
parse_formula("(p,(D,(p,(e)),(p,(e)),(p,(p,(e)))))")

In [13]:
from collections import defaultdict
show_data = defaultdict(list)

In [14]:
for f in forms:
    show_data['form'].append(f)
    show_data['e_counts-mean'].append(df[f"{f}:e_counts"].mean())
    show_data['e_counts-std'].append(df[f"{f}:e_counts"].std())
    show_data['p_counts-mean'].append(df[f"{f}:p_counts"].mean())
    show_data['p_counts-std'].append(df[f"{f}:p_counts"].std())
    show_data['p_depth-mean'].append(df[f"{f}:p_depth"].mean())
    show_data['p_depth-std'].append(df[f"{f}:p_depth"].mean())
    

In [15]:
statdf = pd.DataFrame(show_data)

In [16]:
statdf

,form,e_counts-mean,e_counts-std,p_counts-mean,p_counts-std,p_depth-mean,p_depth-std
0,original,2.884462,0.355806,5.513944,1.429267,2.589641,2.589641
1,DeMorgan,2.884462,0.355806,5.513944,1.429267,2.589641,2.589641
2,DNF,3.123506,0.603892,6.187251,1.867832,2.589641,2.589641
3,diff,2.884462,0.355806,5.513944,1.429267,2.589641,2.589641
4,DNF+diff,3.123506,0.603892,6.187251,1.867832,2.589641,2.589641
5,DNF+MultiIU,3.123506,0.603892,6.187251,1.867832,2.589641,2.589641
6,DNF+MultiIUD,3.123506,0.603892,6.187251,1.867832,2.589641,2.589641


group by the queries by depth and number of queres

In [17]:
for (ec, pc), subdf in df.groupby(["original:e_counts", "original:p_depth"]):
    print(ec, pc, len(subdf[forms]))

1 1 1
1 2 1
1 3 1
2 1 2
2 2 9
2 3 12
3 1 7
3 2 73
3 3 145


In [18]:
for (ec, pc), subdf in df.groupby(["DNF:e_counts", "DNF:p_depth"]):
    print(ec, pc, len(subdf[forms]))

1 1 1
1 2 1
1 3 1
2 1 2
2 2 9
2 3 12
3 1 4
3 2 52
3 3 109
4 1 3
4 2 21
4 3 36
